In [1]:
# imports

## allow interaction with the plots
%matplotlib widget

%load_ext autoreload
%autoreload 2

# add the main directory reference and import 'imports.py'
import sys
import os

if ".." not in sys.path:
    sys.path.insert(0, "..")
from imports import *

logger = logging.getLogger()
logger.setLevel(level=logging.WARN)

__builtins__.verbosity = 4
# Set the HF_HOME environment variable
os.environ["HF_HOME"] = "/run/media/HUNK/DATASETS/HF"


In [2]:
# Declare a few global data/train params
from lib.optim import WarmupCosineLR
from typing import override
from lib.train import *


class params:
    num_workers = 0
    batch_size = 160
    seq_len = 140
    files_per_epoch = 800
    prefetch_factor = (10,)

    max_epochs = 200
    lr = 0.005

    epoch_every = 100


In [23]:
from data.pgnSquare import *


dt = PGNSquareData(PGNSquareDataConfig.create(params))

dt.preview()



Loader 0 (IterableDataset) Preview:
--------------------------------------------------
Constituent shapes:
batch[0]: torch.Size([160, 140]), torch.int64
batch[1]: torch.Size([160, 1]), torch.int64

First 5 samples:

Sample 0: 

tensor([1435, 5328, 1440, 6162, 1010, 6078, 1404, 5298, 1256, 5189,  363, 4097,
        1527, 4368,  777, 5360, 1497, 5139, 1529, 4436,  158, 4331, 1064, 4156,
        1170, 4086, 1076, 5541, 1042, 5949, 2089, 4366, 1373, 5443, 2122, 6106,
        1410, 5295, 1380, 5699,  129, 5355, 1214, 4301,  294, 4942, 3567,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,  

In [ ]:
# Assume `model` is your pretrained transformer model
# Freeze the entire model
for param in model.parameters():
    param.requires_grad = False

# The layer you want to hook into:
target_layer = model.encoder.layers[-1]  # example, adjust as needed

# Add a linear layer for classification
fc = nn.Linear(hidden_dim, num_classes).to(device)

# Forward hook to capture the output of the target layer
activations = None


def hook_fn(m, i, o):
    nonlocal activations
    activations = o.detach()


hook_handle = target_layer.register_forward_hook(hook_fn)

# Optimizer only for fc layer
optimizer = torch.optim.Adam(fc.parameters(), lr=1e-3)

# Training loop
model.eval()  # model is frozen, so eval mode is fine if it matters
fc.train()

for batch in train_loader:
    inputs, labels = batch  # adjust based on your data structure
    optimizer.zero_grad()
    # Forward pass through the model to fill `activations` from the hook
    with torch.no_grad():
        _ = model(inputs.to(device))

    # Now forward pass through the fc layer
    outputs = fc(activations)
    loss = loss_fn(outputs, labels.to(device))
    loss.backward()
    optimizer.step()
